In [1]:
#Przygotowanie środowiska
from kaggle_secrets import UserSecretsClient
import os

user_secrets = UserSecretsClient()
my_token = user_secrets.get_secret("GITHUB_TOKEN")

!git clone https://{my_token}@github.com/WebCrawlerr/GAT.git

# Pobranie zmian
!git pull origin main
# Wyświetla listę plików w pobranym folderze
!ls -R  GAT

import shutil
import os
import sys
# 1. Zmiana katalogu roboczego
os.chdir('./GAT')
print(f"Aktualny katalog: {os.getcwd()}")

# Najpierw sprawdźmy wersję po restarcie (czasem GPU ma starszego PyTorcha)
import torch
version = torch.__version__
print(f"PyTorch version: {version}")

# Instalacja PyG i bibliotek pomocniczych - korzystamy z gotowych paczek .whl
#Ta komenda dynamicznie dobiera link do wersji PyTorcha
torch_version_suffix = version.split('+')[0].replace('.', '') # np. 260 lub 240
cuda_version_suffix = 'cu' + torch.version.cuda.replace('.', '') # np. cu121

print(f"Instaluję dla: torch-{torch_version_suffix} + {cuda_version_suffix}")

# Instalacja zależności
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv \
  -f https://data.pyg.org/whl/torch-{version}.html

# Instalacja reszty z pliku (bez ponownego instalowania torch-geometric)
!pip install -r requirements.txt

import torch
import torch_geometric

print(f"CUDA dostępne: {torch.cuda.is_available()}")
print(f"Wersja PyTorch: {torch.__version__}")
print(f"Wersja PyG: {torch_geometric.__version__}")

# Próba importu kluczowych warstw
try:
    from torch_geometric.nn import GATConv
    print("✅ GATConv zaimportowany pomyślnie!")
except ImportError as e:
    print(f"❌ Błąd importu GATConv: {e}")

# Sprawdzenie czy mamy backendy (scatter/sparse)
# PyG 2.7+ potrafi czasem działać bez nich (wolniej), ale warto sprawdzić
try:
    import torch_scatter
    print("✅ torch_scatter obecny")
except ImportError:
    print("⚠️ torch_scatter BRAK (Może to spowolnić trening lub wywołać błąd)")

try:
    import torch_sparse
    print("✅ torch_sparse obecny")
except ImportError:
    print("⚠️ torch_sparse BRAK (Wymagany dla niektórych operatorów)")

Cloning into 'GAT'...
remote: Enumerating objects: 420, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 420 (delta 85), reused 72 (delta 28), pack-reused 271 (from 1)
Receiving objects: 100% (420/420), 3.52 MiB | 33.11 MiB/s, done.
Resolving deltas: 100% (251/251), done.
fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
GAT:
 evaluate_model.py  "pdf's"			 run_experiments.py
 filter_protein.py   prepare_bindingdb_data.py	 src
 inspect_data.py     prepare_leash_data.py	 tests
 leash_head.csv      __pycache__		 verify_changes.py
 main.py	     README.md
 models		     requirements.txt

GAT/models:
 best_model.pth		'gat-inz (2).ipynb'
 fold_0_best_model.pth	 HSA_gat_h64_ep12_ap0.3126.pth

"GAT/pdf's":
Kacper_Wieczorek___praca_dyplomowa_inżynierska.pdf

GAT/__pycache__:
filter_protein.cpython-312.pyc	prepare_leash_data.cpython-31

In [2]:
#Optymalizacja modelu
#!python main.py --optimize --n_trials 15 --raw_file /kaggle/input/bindingdb-smiles/BindingDB_All.tsv --processed_dir ./data/processed
#--cv 5

In [3]:
#STARE PRZYGOTOWANIE DANYCH BELKA
#!python prepare_leash_data.py --input_file /kaggle/input/leash-BELKA/train.csv --output_dir /kaggle/working/ --protein BRD4 --negative_rate 1

In [4]:
# Uruchomienie Learning Curve
#!python run_experiments.py \
#    --experiment learning_curve \
#    --filtered_file "/kaggle/input/hsa-filtered/leash_brd4_filtered.csv" \
#    --limit 2741784 \
#    --epochs 20

In [5]:
#!python filter_protein.py \
 #   --input_file /kaggle/input/leash-BELKA/train.csv \
  #  --output_file /kaggle/working/leash_seh_filtered.csv \
   # --protein sEH

In [6]:
#--filtered_file /kaggle/input/gat-inz/leash_brd4_filtered.csv \
#!python main.py \
#--filtered_file /kaggle/input/seh-filtered/leash_seh_filtered.csv \
#--processed_dir . \
#--limit 2500000 \
#--cv 0 \
#--target sEH

In [7]:
#!python -m src.inference \
 # --model_path /kaggle/input/nabu-1/pytorch/default/1/best_model_1point8mil_data.pth \
  #--test_file /kaggle/input/leash-BELKA/test.csv \
  #--batch_size 4096 \
  #--output_file submission.csv

In [8]:
#!python -m src.inference_multi \
#    --test_file /kaggle/input/leash-BELKA/test.csv \
#    --model_path_BRD4 /kaggle/input/nabu-1/pytorch/default/1/best_model_1point8mil_data.pth \
#    --model_path_HSA /kaggle/input/hsa/pytorch/default/1/BRD4_gat_h64_ep11_ap0.3123.pth \
#    --model_path_sEH /kaggle/input/seh/pytorch/default/1/sEH_gat_h64_ep22_ap0.8652.pth \
#    --output_file submission.csv

In [9]:
#!python src/recover_results.py

In [10]:
!python main.py --filtered_file /kaggle/input/brd4-filtered/leash_brd4_filtered.csv --processed_dir . --target BRD4 --limit 2500000 --cv 0

Starting GAT BRD4 Binding Prediction Pipeline...
Using provided filtered file: /kaggle/input/brd4-filtered/leash_brd4_filtered.csv
Initializing Graph Dataset in ....
Processing...
Sampling from /kaggle/input/brd4-filtered/leash_brd4_filtered.csv using Polars (Test Mode=False)...
Pass 1: Counting class distribution...
Found: Positives=456964, Negatives=97958646
Target: Positives=456964, Negatives=2043036
Sampling Probabilities: Pos=1.0000, Neg=0.0209
Pass 2: Extracting samples...
Final Dataset: 2499555 samples
Done!
Dataset loaded. Size: 2499555. Columns: ['Ligand SMILES', 'Label', 'buildingblock3_smiles']
Splitting data using Building Blocks...
Performing fast Building Block split using DataFrame...
Split results: Train=1999634, Val=249965, Test=249956
Using device: cuda
Class Distribution in Train: Pos=369532, Neg=1630102
Focal Loss Alpha (Dynamic): 0.8152
Model will be saved as: BRD4_gat_h64_bs64_lr0.0005_data1999k_best.pth
Epoch: 001, Loss: 0.0522, Val AP: 0.1754, Val AUC: 0.5000
 -

In [11]:
#!python evaluate_model.py \
#  --model_path /kaggle/input/nabu-1/pytorch/default/1/best_model_1point8mil_data.pth \
#  --filtered_file /kaggle/input/hsa-filtered/leash_brd4_filtered.csv \
#  --output_dir ./plots \
#  --data_dir .